In [1]:
import random
import numpy as np
import aifc
import csv
import pandas as pd
import sys
import matplotlib.pyplot as plt

def extract_file():
    df = pd.read_csv('train.csv', header=0, sep=',')
    df.columns=['sample', 'label'] 
    return df
    
def pourc(tl, pourc):
    return pourc*tl/100

def compare_our_arrays(x,y):
    score = 0
    for i in x: 
        if x[i] == y[i]:
            score+=1
    return score

def create_label():
    df = extract_file()
    label_list = []
    for i in range(len(df)):
        label_list.append(df['label'][i])
    return label_list

def split_our_lists(): #Splits our lists 
    train = []
    validation = []
    test = []
    label_list = create_label()
    random.shuffle(label_list)
    tl = len(label_list)
    
    for i in range(tl):
        if i < pourc(tl,60):
            train.append(label_list[i])
        elif i < pourc(tl,80):
            validation.append(label_list[i])
        else:
            test.append(label_list[i])
    return train, validation, test


def show_signal(filename): #shows our signal thx to matplotlib
    liste_sample_fft = []
    s = aifc.open('Data/train/' + filename, "r")
    N = s.getnframes()
    strsig = s.readframes(N)
    y = np.fromstring(strsig, np.short).byteswap()
    FE = s.getframerate()
    A = np.fft.fft(y)
    DeltaT = 1./FE
    ampl = np.abs(A)*1./N
    freqs = np.fft.fftfreq(N, DeltaT)
    P = plt.plot(freqs[:int(N/2)], ampl[:int(N/2)]) 
    plt.ylabel("Amplitude")
    plt.xlabel("Frequence")
    plt.show()

def fct_FFT(filename) : #Fast Fourier transformation decomposes our temporal signal into frequency signal
    liste_sample_fft = []
    s = aifc.open('Data/train/' + filename, "r")
    N = s.getnframes()
    strsig = s.readframes(N)
    y = np.fromstring(strsig, np.short).byteswap()
    FE = s.getframerate()
    A = np.fft.fft(y)
    DeltaT = 1./FE
    ampl = np.abs(A)*1./N
    liste_sample_fft.append(ampl)
    return ampl

def Parcours_file_fft(listname): #Allows us to take the name of the files of train or test or validation
    df = extract_file() 
    df_train = []
    for i in range(len(listname)):
        df_train.append(fct_FFT(df["sample"][i]))
    return df_train

liste = split_our_lists() 

from sklearn.ensemble import RandomForestClassifier
ClassifiedDatas = RandomForestClassifier()



In [4]:
Train = liste[0]
k = 800
Train_fft = Parcours_file_fft(liste[0])
ClassifiedDatas.fit(Train_fft[:k], Train[:k])
ClassifiedDatas.score(Train_fft[:k], Train[:k])

Y = np.array(Train[:k])
Y_predicted = ClassifiedDatas.predict(Train_fft[:k])
sum(Y == Y_predicted)

784

In [3]:
# Effectuer avec le Validation

Validation = liste[1]
Validation_fft = Parcours_file_fft(liste[1])
print(ClassifiedDatas.score(Validation_fft[:k], Validation[:k]))

Xtst = np.array(Validation[:k])
Ytst = ClassifiedDatas.predict(Validation_fft[:k])

print(sum(Xtst == Ytst))

0.67125
537


In [83]:
# ??  
print(ClassifiedDatas.predict_proba(vect_tot[:15]))
ClassifiedDatas.decision_path(vect_tot[:15])
# Path of the tree to each value
ClassifiedDatas.apply(vect_tot[:15])


[[ 0.1  0.9]
 [ 0.6  0.4]
 [ 0.8  0.2]
 [ 0.9  0.1]
 [ 0.4  0.6]
 [ 0.8  0.2]
 [ 0.   1. ]
 [ 0.9  0.1]
 [ 0.9  0.1]
 [ 0.5  0.5]
 [ 0.9  0.1]
 [ 1.   0. ]
 [ 1.   0. ]
 [ 1.   0. ]
 [ 0.9  0.1]]


array([[12,  3, 14,  4, 15, 13, 17, 15, 23,  1],
       [ 8,  3,  3, 12, 10,  5,  5, 12,  4,  4],
       [ 8,  9,  9,  4, 10,  8, 11, 11, 15,  5],
       [14, 16,  6, 10, 10,  8,  5, 12, 15, 13],
       [ 9, 16,  9,  9, 17, 11, 17, 15, 19, 13],
       [ 9, 20, 16, 10, 10,  8, 11, 15, 15, 13],
       [ 9, 20,  3, 16, 17, 11, 17, 18, 26, 15],
       [ 8, 16,  9,  9, 10,  8,  5, 15, 15, 13],
       [13, 16,  9, 12, 10,  8, 17, 15, 15, 13],
       [ 6, 10, 11, 12, 14,  9,  5, 16, 15, 13],
       [ 8, 16,  6, 12, 10,  8,  5, 15, 15, 13],
       [ 8, 16, 16, 10, 10,  8, 11, 15, 15, 13],
       [14, 16,  9,  5, 14,  8, 11, 15, 15,  5],
       [14, 16, 16, 12, 10,  8,  5, 15, 25, 13],
       [14, 16, 16, 12, 10,  8, 11,  2, 15, 13]])